### Parallel Max (Python) [6 points]

Python has a built-in function `max` that iterates over all elements of a collection and returns its maximum, for example:

In [236]:
max([(i * i) % 100 for i in range(1000)])

96

We like to speed up the `max` function when applied to large lists by using two threads: one determines the maximum in the lower half and one in the upper half. The function `parallelmax` first creates these two threads, then waits for them to terminate, then determines which of the two found the largest value and returns that:

In [237]:
from threading import Thread

class Max(Thread):
    def __init__(self, a, l, u):
        super().__init__(); self.a, self.l, self.u = a, l, u
    def run(self):
        self.max = max(self.a[i] for i in range(self.l, self.u))

def parallelmax(a):
    n = len(a)
    m0 = Max(a, 0, n // 2); m1 = Max(a, n // 2, n)
    m0.start(); m1.start()
    m0.join(); m1.join()
    return max(m0.max, m1.max)

In [238]:
parallelmax([1, 5, 3])

5

In [239]:
parallelmax([(i * i) % 100 for i in range(100000)])

96

This is an example of *embarrassing parallelism*: the two threads do not communicate with each other, they are completely independent and only produce a result.

The task is now the generalize `parallelmax` to use `p` threads instead of just two threads with each thread determining the maximum of approximately `n / p` elements of a list with `n` elements. For this, each of the `p` threads will store the maximum of its sublist in a new list and then the maximum of this list needs to be determined. You can assume that `p` is a "small" number and a sequential algorithm for determining that maximum is adequate. In the template below, `p` is an optional argument with default value `2`. Note that `n` may or may not be divisible by `p`:

In [240]:
def parallelmax(a, p = 2):
    nums = len(a)
    if(p >= nums): #if there are more threads than numbers we will use default 2 threads
        p = 2

    threads = []
    maxValues = []
    perThread = nums//p

    for i in range(p):
        lower = i * perThread
        if i < (p-1):
            upper = (i+1)*perThread
        else: #add the remainder to the final thread
            upper = nums
            
        thread = Max(a, lower, upper)
        thread.start()
        threads.append(thread)
        

    for thread in threads:
        thread.join()
        maxValues.append(thread.max)

    return max(maxValues)
    
    
    

In [241]:
assert parallelmax([1, 5, 3]) == 5

In [242]:
a = [(i * i) % 100 for i in range(1000)] # list with 1000 "random" values
assert parallelmax(a, 4) == 96

In [243]:
a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,22,24,25,26,27,28,29,30,31,32,33,34,35,36,34,54,65,23,565,32,46,413,23]
assert parallelmax(a, 7) == 565

In [244]:
a = [126,2,3,45,46,7,3,4,5,6,4,32,21,45,65,120,125]   
assert parallelmax(a,7) == 126